In [29]:
# ====================================================================
# PROJET BUSINESS INTELLIGENCE - TECHSTORE
# ETL PIPELINE - PARTIE MEMBRE 1 : DATA EXTRACTION
# ====================================================================

# %% [markdown]
# # 📊 Extraction des Données (Membre 1)
# 
# Ce notebook contient la partie extraction du pipeline ETL.
# **Responsable :** Membre 1 - Data Extraction Engineer
# 
# ## Objectifs :
# 1. ✅ Extraire les données de MySQL (ERP)
# 2. ✅ Scraper les prix des concurrents
# 3. ✅ Extraire les factures legacy (OCR - BONUS)
# 4. ✅ Valider la qualité des données extraites

# %% Imports pour tout le pipeline ETL
# Installer les dépendances manquantes (exécuter uniquement dans le notebook)


# === Imports de base ===
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

# === Partie 1 : Extraction (Membre 1) ===
import mysql.connector
import requests
from bs4 import BeautifulSoup
import os
import sys
import time
from datetime import datetime

# === Partie 2 : Transformations (Membre 2 - Toi) ===
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pytesseract
from PIL import Image
import re
from fuzzywuzzy import process  # Pour le matching des prix concurrents

print("✅ Bibliothèques importées avec succès")
print(f"📅 Date d'exécution: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# %% [markdown]
# ## 1️⃣ Configuration de la Connexion MySQL

# %% Configuration
MYSQL_CONFIG = {
    'host': 'boughida.com',
    'database': 'techstore_erp',
    'user': 'student_user_4ing',
    'password': 'bi_guelma_2025'
}

# Créer les répertoires nécessaires
os.makedirs('data/extracted', exist_ok=True)
os.makedirs('data/legacy_invoices', exist_ok=True)

print("📋 Configuration chargée")
print(f"   Serveur: {MYSQL_CONFIG['host']}")
print(f"   Base de données: {MYSQL_CONFIG['database']}")

# %% [markdown]
# ## 2️⃣ Connexion et Test

# %% Test de connexion
def test_connection():
    """Tester la connexion à MySQL"""
    try:
        conn = mysql.connector.connect(**MYSQL_CONFIG)
        if conn.is_connected():
            print("✅ Connexion MySQL réussie!")
            
            # Tester une requête simple
            cursor = conn.cursor()
            cursor.execute("SELECT DATABASE()")
            db_name = cursor.fetchone()[0]
            print(f"   Base de données active: {db_name}")
            
            # Lister les tables disponibles
            cursor.execute("SHOW TABLES")
            tables = cursor.fetchall()
            print(f"   Nombre de tables: {len(tables)}")
            
            conn.close()
            return True
    except Exception as e:
        print(f"❌ Erreur de connexion: {e}")
        return False

# Exécuter le test
test_connection()

# %% [markdown]
# ## 3️⃣ Extraction des Tables MySQL

# %% Fonction d'extraction
def extract_mysql_table(table_name, conn):
    """Extraire une table MySQL vers DataFrame"""
    try:
        print(f"📊 Extraction: {table_name}...", end=" ")
        
        query = f"SELECT * FROM {table_name}"
        df = pd.read_sql(query, conn)
        
        # Sauvegarder en CSV
        filename = f"data/extracted/{table_name.replace('table_', '')}.csv"
        df.to_csv(filename, index=False, encoding='utf-8')
        
        print(f"✅ {len(df)} lignes | {len(df.columns)} colonnes")
        
        return df
        
    except Exception as e:
        print(f"❌ Erreur: {e}")
        return None

# %% Extraction de toutes les tables
print("\n" + "="*60)
print("🚀 EXTRACTION DES TABLES MySQL")
print("="*60 + "\n")

# Se connecter
conn = mysql.connector.connect(**MYSQL_CONFIG)

# Liste des tables à extraire
tables_to_extract = [
    'table_sales',
    'table_products',
    'table_reviews',
    'table_customers',
    'table_stores',
    'table_cities',
    'table_categories',
    'table_subcategories'
]

# Dictionnaire pour stocker les DataFrames
dataframes = {}

# Extraire chaque table
for table in tables_to_extract:
    df = extract_mysql_table(table, conn)
    if df is not None:
        # Enlever le préfixe "table_" pour le nom
        clean_name = table.replace('table_', '')
        dataframes[clean_name] = df

# Fermer la connexion
conn.close()

print("\n✅ Extraction MySQL terminée!")
print(f"📦 {len(dataframes)} tables extraites")

# %% [markdown]
# ## 4️⃣ Aperçu des Données Extraites

# %% Afficher un aperçu
print("\n" + "="*60)
print("📋 APERÇU DES DONNÉES EXTRAITES")
print("="*60 + "\n")

for name, df in dataframes.items():
    print(f"\n📊 Table: {name}")
    print(f"   Dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes")
    print(f"   Colonnes: {', '.join(df.columns.tolist()[:5])}...")
    print(f"   Aperçu:")
    print(df.head(3))
    print("-" * 60)

# %% [markdown]
# ## 5️⃣ Statistiques des Ventes

# %% Analyse rapide des ventes
df_sales = dataframes['sales']

print("\n📊 STATISTIQUES DES VENTES")
print("="*60)
print(f"Nombre total de ventes: {len(df_sales):,}")
print(f"Revenu total: {df_sales['Total_Revenue'].sum():,.2f} DZD")
print(f"Revenu moyen par vente: {df_sales['Total_Revenue'].mean():,.2f} DZD")
print(f"Période: {df_sales['Date'].min()} → {df_sales['Date'].max()}")

# %% [markdown]
# ## 6️⃣ Web Scraping - Prix Concurrents

# %% Import du module de scraping
print("\n" + "="*60)
print("🕷️  WEB SCRAPING - PRIX CONCURRENTS")
print("="*60 + "\n")

# Ajouter le répertoire scripts au path pour l'import
sys.path.insert(0, 'scripts')

try:
    from scrape_competitors import scrape_with_fallback
    
    # Exécuter le scraping
    df_competitor = scrape_with_fallback()
    
    if df_competitor is not None:
        print(f"\n✅ Scraping terminé: {len(df_competitor)} produits extraits")
        print(f"💾 Fichier sauvegardé: data/extracted/competitor_prices.csv")
        
        # Afficher un aperçu
        print("\n📋 Aperçu des prix concurrents:")
        print(df_competitor.head(10))
        
        # Ajouter au dictionnaire
        dataframes['competitor_prices'] = df_competitor
    else:
        print("⚠️ Aucune donnée de concurrent récupérée")
        
except Exception as e:
    print(f"❌ Erreur lors du scraping: {e}")
    print("📝 Le module de scraping doit être disponible dans scripts/scrape_competitors.py")

# %% [markdown]
# ## 7️⃣ Extraction des Factures Legacy (OCR - BONUS)

# %% OCR Processing
print("\n" + "="*60)
print("📄 EXTRACTION DES FACTURES LEGACY (OCR - BONUS)")
print("="*60 + "\n")

# Check if invoice images exist
invoice_dir = 'data/legacy_invoices'
has_invoices = os.path.exists(invoice_dir) and len([f for f in os.listdir(invoice_dir) 
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))]) > 0

if has_invoices:
    print("📁 Factures détectées, lancement de l'OCR...")
    
    try:
        from extract_legacy_invoices import InvoiceOCRProcessor
        
        # Initialize and run OCR processor
        processor = InvoiceOCRProcessor(invoice_dir)
        df_legacy = processor.process_and_save()
        
        if df_legacy is not None and len(df_legacy) > 0:
            print(f"\n✅ OCR terminé: {len(df_legacy)} factures traitées")
            print(f"💾 Fichier sauvegardé: data/extracted/legacy_sales.csv")
            
            # Display preview
            print("\n📋 Aperçu des ventes legacy:")
            print(df_legacy.head())
            
            # Add to dataframes dictionary
            dataframes['legacy_sales'] = df_legacy
            
        else:
            print("⚠️ Aucune donnée extraite par OCR")
            
    except ImportError:
        print("⚠️ Module OCR non trouvé (scripts/extract_legacy_invoices.py)")
        print("   Cette étape est optionnelle (BONUS)")
    except Exception as e:
        print(f"⚠️ Erreur OCR: {e}")
        print("   Utilisation de données manuelles à la place...")
        
        try:
            from extract_legacy_invoices import create_manual_data
            df_legacy = create_manual_data()
            dataframes['legacy_sales'] = df_legacy
            print("✅ Données manuelles créées")
        except:
            print("   Passage de cette étape")
else:
    print("ℹ️  Aucune facture trouvée dans data/legacy_invoices/")
    print("   Cette étape est optionnelle (BONUS)")
    print("   Pour activer: placez les images .jpg dans data/legacy_invoices/")

# %% [markdown]
# ## 8️⃣ Validation des Données Extraites

# %% Vérifications de qualité
print("\n" + "="*60)
print("✅ VALIDATION DES DONNÉES")
print("="*60 + "\n")

validation_results = []

for name, df in dataframes.items():
    # Vérifier les valeurs manquantes
    missing = df.isnull().sum().sum()
    missing_pct = (missing / (df.shape[0] * df.shape[1])) * 100
    
    # Vérifier les doublons
    duplicates = df.duplicated().sum()
    
    validation_results.append({
        'Table': name,
        'Lignes': len(df),
        'Colonnes': len(df.columns),
        'Valeurs_Manquantes': missing,
        'Pct_Manquant': f"{missing_pct:.2f}%",
        'Doublons': duplicates,
        'Statut': '✅' if missing_pct < 5 and duplicates < 10 else '⚠️'
    })

df_validation = pd.DataFrame(validation_results)
print(df_validation.to_string(index=False))

# %% [markdown]
# ## 9️⃣ Export Summary Report

# %% Generate extraction summary
print("\n" + "="*60)
print("📊 GÉNÉRATION DU RAPPORT D'EXTRACTION")
print("="*60 + "\n")

summary = {
    'extraction_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_tables': len(dataframes),
    'total_records': sum(len(df) for df in dataframes.values()),
    'files_created': []
}

# List all created files with sizes
for file in sorted(os.listdir('data/extracted')):
    file_path = f"data/extracted/{file}"
    file_size = os.path.getsize(file_path) / 1024  # KB
    summary['files_created'].append({
        'filename': file,
        'size_kb': round(file_size, 2),
        'records': len(pd.read_csv(file_path))
    })

# Save summary to JSON
import json
with open('data/extracted/extraction_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✅ Rapport d'extraction sauvegardé: extraction_summary.json")

# %% [markdown]
# ## 🔟 Résumé de l'Extraction

# %% Résumé final
print("\n" + "="*70)
print("📊 RÉSUMÉ DE L'EXTRACTION (MEMBRE 1)")
print("="*70 + "\n")

print("✅ TÂCHES COMPLÉTÉES:")
print("   1. ✅ Connexion MySQL établie et testée")
print(f"   2. ✅ {len([k for k in dataframes.keys() if k not in ['competitor_prices', 'legacy_sales']])} tables extraites de l'ERP")
print("   3. ✅ Web scraping des prix concurrents effectué")

if 'legacy_sales' in dataframes:
    print("   4. ✅ Extraction OCR des factures legacy (BONUS)")
else:
    print("   4. ⏭️  Extraction OCR non effectuée (optionnel)")

print("   5. ✅ Toutes les données sauvegardées en CSV")
print("   6. ✅ Validation de la qualité des données effectuée")

print("\n📦 FICHIERS CRÉÉS:")
for file_info in summary['files_created']:
    print(f"   • {file_info['filename']:<30} ({file_info['size_kb']:>8.1f} KB) - {file_info['records']:>6,} lignes")

print(f"\n📊 STATISTIQUES GLOBALES:")
print(f"   • Total de lignes extraites: {summary['total_records']:,}")
print(f"   • Nombre de fichiers: {len(summary['files_created'])}")
print(f"   • Date d'extraction: {summary['extraction_date']}")

print("\n🚀 PROCHAINE ÉTAPE:")
print("   → Membre 2 peut maintenant transformer ces données")
print("   → Fichiers disponibles dans: data/extracted/")
print("   → Rapport disponible dans: data/extracted/extraction_summary.json")

print("\n💡 NOTES POUR L'ÉQUIPE:")
print("   • Les données ERP couvrent la période 2023-2025")
print("   • Les prix concurrents sont à jour à la date d'extraction")
if 'legacy_sales' in dataframes:
    print("   • Les factures legacy (2022) ont été numérisées (BONUS)")

print("\n" + "="*70)
print("✅ EXTRACTION TERMINÉE AVEC SUCCÈS")
print("="*70)

# %% [markdown]
# ## 📝 Notes de Développement
# 
# ### Structure des fichiers créés:
# - `sales.csv`: Transactions de vente (OLTP)
# - `products.csv`: Catalogue produits
# - `customers.csv`: Profils clients
# - `stores.csv`: Magasins
# - `cities.csv`: Géographie
# - `categories.csv` & `subcategories.csv`: Hiérarchie produits
# - `reviews.csv`: Avis clients (pour sentiment analysis)
# - `competitor_prices.csv`: Prix concurrents (web scraping)
# - `legacy_sales.csv`: Ventes 2022 (OCR - optionnel)
# 
# ### Prochaines étapes (Membre 2):
# - Charger ces CSV
# - Appliquer les transformations (nettoyage, enrichissement)
# - Calculer Net_Profit avec les fichiers Excel
# - Effectuer l'analyse de sentiment (VADER)
# - Préparer les données pour le Data Warehouse

✅ Bibliothèques importées avec succès
📅 Date d'exécution: 2025-12-26 21:14:50
📋 Configuration chargée
   Serveur: boughida.com
   Base de données: techstore_erp
✅ Connexion MySQL réussie!
   Base de données active: techstore_erp
   Nombre de tables: 8

🚀 EXTRACTION DES TABLES MySQL

📊 Extraction: table_sales... ✅ 25000 lignes | 7 colonnes
📊 Extraction: table_products... ✅ 38 lignes | 5 colonnes
📊 Extraction: table_reviews... ✅ 3000 lignes | 5 colonnes
📊 Extraction: table_customers... ✅ 1200 lignes | 3 colonnes
📊 Extraction: table_stores... ✅ 12 lignes | 3 colonnes
📊 Extraction: table_cities... ✅ 12 lignes | 3 colonnes
📊 Extraction: table_categories... ✅ 5 lignes | 2 colonnes
📊 Extraction: table_subcategories... ✅ 15 lignes | 3 colonnes

✅ Extraction MySQL terminée!
📦 8 tables extraites

📋 APERÇU DES DONNÉES EXTRAITES


📊 Table: sales
   Dimensions: 25000 lignes × 7 colonnes
   Colonnes: Trans_ID, Date, Store_ID, Product_ID, Customer_ID...
   Aperçu:


,Trans_ID,Date,Store_ID,Product_ID,Customer_ID,Quantity,Total_Revenue
0,1,2024-07-02 17:15:00,3,P125,C0200,2,9000.0
1,2,2023-07-01 13:52:00,11,P136,C0883,1,2500.0
2,3,2025-01-05 18:56:00,7,P106,C0669,1,320000.0


------------------------------------------------------------

📊 Table: products
   Dimensions: 38 lignes × 5 colonnes
   Colonnes: Product_ID, Product_Name, SubCat_ID, Unit_Price, Unit_Cost...
   Aperçu:


,Product_ID,Product_Name,SubCat_ID,Unit_Price,Unit_Cost
0,P100,HP Victus 15,1,125000.0,87901.0
1,P101,Dell XPS 13,1,260000.0,167880.0
2,P102,MacBook Air M2,1,195000.0,141052.0


------------------------------------------------------------

📊 Table: reviews
   Dimensions: 3000 lignes × 5 colonnes
   Colonnes: Review_ID, Product_ID, Customer_ID, Rating, Review_Text...
   Aperçu:


...

------------------------------------------------------------

📊 STATISTIQUES DES VENTES
Nombre total de ventes: 25,000
Revenu total: 859,661,700.00 DZD
Revenu moyen par vente: 34,386.47 DZD
Période: 2023-01-01 09:43:00 → 2025-12-30 20:58:00

🕷️  WEB SCRAPING - PRIX CONCURRENTS

📡 Connexion à: https://boughida.com/competitor/
🔍 0 produits trouvés

✅ Scraping terminé: 0 produits extraits
💾 Fichier sauvegardé: data/extracted/competitor_prices.csv

📋 Aperçu des prix concurrents:


""



✅ VALIDATION DES DONNÉES



,Table,Lignes,Colonnes,Valeurs_Manquantes,Pct_Manquant,Doublons,Statut
0,sales,25000,7,0,0.00%,0,✅
1,products,38,5,0,0.00%,0,✅
2,reviews,3000,5,0,0.00%,0,✅
3,customers,1200,3,0,0.00%,0,✅
4,stores,12,3,0,0.00%,0,✅
5,cities,12,3,0,0.00%,0,✅
6,categories,5,2,0,0.00%,0,✅
7,subcategories,15,3,0,0.00%,0,✅



📊 RÉSUMÉ DE L'EXTRACTION (MEMBRE 1)

✅ TÂCHES COMPLÉTÉES:
   1. Connexion MySQL établie et testée
   2. 8 tables extraites de l'ERP
   3. Web scraping des prix concurrents effectué
   4. Toutes les données sauvegardées en CSV
   5. Validation de la qualité des données effectuée

📦 FICHIERS CRÉÉS:
   • categories.csv (0.1 KB)
   • cities.csv (0.2 KB)
   • competitor_prices.csv (0.0 KB)
   • customers.csv (26.9 KB)
   • products.csv (1.5 KB)
   • reviews.csv (179.5 KB)
   • sales.csv (1199.7 KB)
   • stores.csv (0.4 KB)
   • subcategories.csv (0.3 KB)

🚀 PROCHAINE ÉTAPE:
   → Membre 2 peut maintenant transformer ces données
   → Fichiers disponibles dans: data/extracted/

✅ EXTRACTION TERMINÉE AVEC SUCCÈS
PARTIE 2 – TRANSFORMATION (MEMBRE 2) + CRÉATION DIM TABLES
📂 Projet racine   : e:\BI\project\TechStore
📥 Lecture depuis  : e:\BI\project\TechStore\data\extracted
📤 Écriture dans   : e:\BI\project\TechStore\data\transformed
---------------------------------------------------------------